In [25]:
%load_ext autoreload
%autoreload 2

In [26]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [27]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-06 17:00:00+00:00


In [28]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [29]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-06 11:21:05,262 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 11:21:05,278 INFO: Initializing external client
2025-03-06 11:21:05,278 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 11:21:06,167 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214708


In [30]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.39s) 


In [31]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,212,0.0,2025-03-06 08:00:00+00:00
1,52,0.0,2025-03-06 08:00:00+00:00
2,129,1.0,2025-03-06 08:00:00+00:00
3,145,-0.0,2025-03-06 08:00:00+00:00
4,140,51.0,2025-03-06 08:00:00+00:00
...,...,...,...
499,16,0.0,2025-03-06 17:00:00+00:00
500,174,0.0,2025-03-06 17:00:00+00:00
501,258,0.0,2025-03-06 17:00:00+00:00
502,131,0.0,2025-03-06 17:00:00+00:00


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  504 non-null    int32                  
 1   predicted_demand    504 non-null    float64                
 2   pickup_hour         504 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 10.0 KB


In [33]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-06 11:21:12,749 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 11:21:12,767 INFO: Initializing external client
2025-03-06 11:21:12,770 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 11:21:13,418 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214708
Current UTC time: 2025-03-06 16:21:12.749015+00:00
Next hour: 2025-03-06 17:00:00+00:00
Querying for date range: 2025-03-06 to 2025-03-07
Filtering for hour: 2025-03-06 17:00
Found 252 records


In [34]:
now = datetime.now(timezone.utc)

In [35]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
252,118,0.0,2025-03-06 17:00:00+00:00
253,59,0.0,2025-03-06 17:00:00+00:00
254,217,0.0,2025-03-06 17:00:00+00:00
255,175,0.0,2025-03-06 17:00:00+00:00
256,88,2.0,2025-03-06 17:00:00+00:00
...,...,...,...
499,16,0.0,2025-03-06 17:00:00+00:00
500,174,0.0,2025-03-06 17:00:00+00:00
501,258,0.0,2025-03-06 17:00:00+00:00
502,131,0.0,2025-03-06 17:00:00+00:00


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  504 non-null    int32                  
 1   predicted_demand    504 non-null    float64                
 2   pickup_hour         504 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 10.0 KB


In [37]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [38]:
dt = current_date.ceil('h')


In [39]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-06 11:21:14,993 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 11:21:14,999 INFO: Initializing external client
2025-03-06 11:21:15,000 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 11:21:15,670 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214708
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.99s) 


In [40]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.34s) 


,pickup_location_id,predicted_demand,pickup_hour


In [41]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,212,0.0,2025-03-06 08:00:00+00:00
1,52,0.0,2025-03-06 08:00:00+00:00
2,129,1.0,2025-03-06 08:00:00+00:00
3,145,-0.0,2025-03-06 08:00:00+00:00
4,140,51.0,2025-03-06 08:00:00+00:00
...,...,...,...
499,16,0.0,2025-03-06 17:00:00+00:00
500,174,0.0,2025-03-06 17:00:00+00:00
501,258,0.0,2025-03-06 17:00:00+00:00
502,131,0.0,2025-03-06 17:00:00+00:00


In [42]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [43]:
predictions = fetch_next_hour_predictions()

2025-03-06 11:21:27,399 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 11:21:27,413 INFO: Initializing external client
2025-03-06 11:21:27,413 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 11:21:28,119 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214708
Current UTC time: 2025-03-06 16:21:27.399678+00:00
Next hour: 2025-03-06 17:00:00+00:00
Found 252 records


In [44]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-06 11:21:29,022 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 11:21:29,022 INFO: Initializing external client
2025-03-06 11:21:29,022 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 11:21:29,684 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214708


In [45]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.14s) 


In [46]:
df["pickup_hour"].max()

Timestamp('2025-03-06 17:00:00+0000', tz='Etc/UTC')

In [47]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-06 17:00:00+00:00


In [48]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
252,118,0.0,2025-03-06 17:00:00+00:00
253,59,0.0,2025-03-06 17:00:00+00:00
254,217,0.0,2025-03-06 17:00:00+00:00
255,175,0.0,2025-03-06 17:00:00+00:00
256,88,2.0,2025-03-06 17:00:00+00:00
...,...,...,...
499,16,0.0,2025-03-06 17:00:00+00:00
500,174,0.0,2025-03-06 17:00:00+00:00
501,258,0.0,2025-03-06 17:00:00+00:00
502,131,0.0,2025-03-06 17:00:00+00:00
